In [1]:
#import libraries
import time
import requests 
from bs4 import BeautifulSoup
import pandas as pd

#lists needed
listlinks=[]
listtitles=[]
listreviews=[]
listprices=[]
listdetails=[]
listnum_ratings_reviews=[]
listdiscounts=[]


In [ ]:
'''If you're encountering a 429 response ("Too Many Requests") when scraping multiple pages of a website, it likely means that the website has rate limits in place to prevent abuse or overloading their server. To solve this problem, you can implement techniques to mitigate the rate limiting and allow your scraping process to proceed smoothly. Here are some approaches you can try:

Add delays between requests: Introduce a delay between each request to the website. This can help prevent triggering rate limits. You can use Python's time.sleep() function to pause execution for a specified number of seconds between requests.

Randomize the delays: Instead of using a fixed delay, add some randomness to the delay time. This can make your scraping pattern less predictable and reduce the likelihood of hitting rate limits.

Use rotating proxies: Rotate through a pool of proxies for each request. This helps distribute your requests across different IP addresses, making it harder for the website to identify and block your scraper.

Modify user-agent: Set a user-agent header in your requests to mimic a real web browser. Some websites might be more lenient towards requests coming from common browsers.

Check for API: If the website provides an API for accessing the data you need, consider using it instead of scraping the HTML. APIs often have higher rate limits and are more reliable for accessing structured data.

Respect robots.txt: Ensure that your scraping script follows the guidelines specified in the website's robots.txt file. This file often specifies rules for web crawlers, including allowed/disallowed paths and crawl rate limits.

Here's a basic example demonstrating how to add a delay between requests using time.sleep():'''

In [2]:

for i in range(1,11):
    url=f'https://www.flipkart.com/search?q=mobiles&as=on&as-show=on&otracker=AS_Query_TrendingAutoSuggest_1_0_na_na_na&otracker1=AS_Query_TrendingAutoSuggest_1_0_na_na_na&as-pos=1&as-type=TRENDING&suggestionId=mobiles&requestId=cb397401-5757-4371-af9d-8c7094821d5b&page={i}'
    
    #get page content
    content=requests.get(url)
    
    #check if the response fails and solve that problem and make sleep to delay the response to avoid the fails
    while (content.status_code == 429):
        
        content=requests.get(url)
        time.sleep(5)
    time.sleep(5)
    #soup
    soup=BeautifulSoup(content.text,'lxml')
    
    page=soup.find_all('div',class_='col col-7-12')
    
    #extract titles of mobiles
    mobtitles=soup.find_all('div',class_='_4rR01T')
    for title in mobtitles:
        listtitles.append(title.text)
        
        
        
    #extract prices 
    prices=soup.find_all('div',class_='_30jeq3 _1_WHN1')
    for price in prices:
        listprices.append(price.text)  
        
        
        
    #extract discounts
    discounts=soup.find('div',class_='_1YokD2 _3Mn1Gg').find_all('div',class_='_3Ay6Sb')
    for discount in discounts:
        listdiscounts.append(discount.text) 
        
    
    '''extract reviews ----> here there was an issue because we need to scrape reviews in the big box in the page
    and not the entire rating in the whole page'''

    reviews=soup.find('div',class_='_1YokD2 _3Mn1Gg').find_all('div',class_='_3LWZlK')
    for review in reviews:
        listreviews.append(review.text) 

        
    #extract details
    details=soup.find_all('ul',class_='_1xgFaf')
    for i in details:
        listdetails.append(i.text) 
        
    #extract ratings & reviews
    mobratings_reviews=soup.find_all('span',class_='_2_R_DZ')
    for i in mobratings_reviews:
        listnum_ratings_reviews.append(i.text)


In [3]:
#check lists length
print(len(listtitles))
print(len(listreviews))
print(len(listprices))
print(len(listdetails))
print(len(listnum_ratings_reviews))

240
240
240
240
240
239


In [4]:
 #make a framework
df=pd.DataFrame({"Mobile_name":listtitles,"Details":listdetails,"Price":listprices,"Reviews":listreviews,"Num_Ratings_Reviews":listnum_ratings_reviews})


#split num_ratings& num reviews
df[['Num_Ratings', 'Num_Reviews']] = df['Num_Ratings_Reviews'].str.split('&', 1, expand=True)
df.drop(columns=['Num_Ratings_Reviews'], inplace=True)

In [5]:
df

,Mobile_name,Details,Price,Reviews,Num_Ratings,Num_Reviews
0,"Apple iPhone 15 (Blue, 128 GB)",128 GB ROM15.49 cm (6.1 inch) Super Retina XDR...,"₹65,999",4.6,"13,968 Ratings",958 Reviews
1,"Motorola G34 5G (Ocean Green, 128 GB)",8 GB RAM | 128 GB ROM16.51 cm (6.5 inch) HD+ D...,"₹11,999",4.2,"6,481 Ratings",899 Reviews
2,"vivo T2 5G (Nitro Blaze, 128 GB)",8 GB RAM | 128 GB ROM16.21 cm (6.38 inch) Full...,"₹17,999",4.3,"16,645 Ratings","1,170 Reviews"
3,"vivo T2 5G (Velocity Wave, 128 GB)",6 GB RAM | 128 GB ROM16.21 cm (6.38 inch) Full...,"₹15,999",4.4,"95,259 Ratings","6,250 Reviews"
4,"vivo T2 5G (Nitro Blaze, 128 GB)",6 GB RAM | 128 GB ROM16.21 cm (6.38 inch) Full...,"₹15,999",4.4,"95,259 Ratings","6,250 Reviews"
...,...,...,...,...,...,...
235,"SAMSUNG Galaxy M14 5G (Smoky Teal, 128 GB)",4 GB RAM | 128 GB ROM16.76 cm (6.6 inch) Displ...,"₹11,255",4.2,"3,758 Ratings",233 Reviews
236,Samsung Galaxy S21 FE 5G with Snapdragon 888 (...,8 GB RAM | 128 GB ROM16.26 cm (6.4 inch) Full ...,"₹29,999",4.3,"1,29,904 Ratings","12,598 Reviews"
237,SAMSUNG Guru Music 2,0.53 RAM | NA KB ROM | Expandable Upto 16 GB5....,"₹1,268",3.8,"4,117 Ratings",273 Reviews
238,SAMSUNG Metro 313 Dual Sim,16 MB RAM | 2 MB ROM5.08 cm (2 inch) Display0....,"₹2,340",4.2,"15,050 Ratings","1,482 Reviews"


In [8]:
df.to_csv(r"G:\education\python\web scraping\data scraping csv\E-commerce-Mobile-Phone-Scraper.csv")